[example apis](https://www.here.com/docs/bundle/data-sdk-for-python-developer-guide-v2/page/topics/notebooks.html)

### try with api

In [3]:
import time #To generate the OAuth timestamp
import urllib.parse #To URLencode the parameter string
import hmac #To implement HMAC algorithm
import hashlib #To generate SHA256 digest
from base64 import b64encode #To encode binary data into Base64
import binascii #To convert data into ASCII
import requests #To make HTTP requests
import os
from here_location_services import LS
import pandas
from here_location_services import PlatformCredentials
import json

### Example 1

In [13]:
LS_API_KEY = os.environ.get("LS_API_KEY")  # Get API KEY from environment.
ls = LS(api_key="coeSXcu16kjdJAujw9INxZRAp_U")


NameError: name 'os' is not defined

In [35]:
address = "Invalidenstr 116, 10115 Berlin, Germany"
geo = ls.geocode(query=address)
print(json.dumps(geo.to_geojson(), indent=2, sort_keys=True))

SSLError: HTTPSConnectionPool(host='geocode.search.hereapi.com', port=443): Max retries exceeded with url: /v1/geocode?q=Invalidenstr+116%2C+10115+Berlin%2C+Germany&limit=20&lang=en-US&apiKey=coeSXcu16kjdJAujw9INxZRAp_U (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1006)')))

### Example 2

In [4]:
grant_type = 'client_credentials'
oauth_consumer_key = 'ZvObrUoexEwKC3KgiMuy3w' #From credentials.properties file
oauth_nonce = str(int(time.time()*1000))
oauth_signature_method = 'HMAC-SHA256'
oauth_timestamp = str(int(time.time()))
oauth_version = '1.0'

In [5]:
def create_parameter_string(grant_type, oauth_consumer_key,oauth_nonce,oauth_signature_method,oauth_timestamp,oauth_version):
    parameter_string = ''
    parameter_string = parameter_string + 'grant_type=' + grant_type
    parameter_string = parameter_string + '&oauth_consumer_key=' + oauth_consumer_key
    parameter_string = parameter_string + '&oauth_nonce=' + oauth_nonce
    parameter_string = parameter_string + '&oauth_signature_method=' + oauth_signature_method
    parameter_string = parameter_string + '&oauth_timestamp=' + oauth_timestamp
    parameter_string = parameter_string + '&oauth_version=' + oauth_version
    return parameter_string

parameter_string = create_parameter_string(grant_type, oauth_consumer_key,oauth_nonce,oauth_signature_method,oauth_timestamp,oauth_version)
encoded_parameter_string = urllib.parse.quote(parameter_string, safe='')

In [6]:
url = 'https://account.api.here.com/oauth2/token'
encoded_base_string = 'POST' + '&' + urllib.parse.quote(url, safe='')
encoded_base_string = encoded_base_string + '&' + encoded_parameter_string

In [7]:
access_key_secret = 'mkupOBOzTREwAaqFwIsr9Z0Cmax8FV_YRKYiDY1SBlZ1FYfjI7QOtcCcy4D-d2rpdlxrncRAHFwvp-Zu4wgClg' #From credentials.properties file
signing_key = access_key_secret + '&'

def create_signature(secret_key, signature_base_string):
    encoded_string = signature_base_string.encode()
    encoded_key = secret_key.encode()
    temp = hmac.new(encoded_key, encoded_string, hashlib.sha256).hexdigest()
    byte_array = b64encode(binascii.unhexlify(temp))
    return byte_array.decode()

oauth_signature = create_signature(signing_key, encoded_base_string)
encoded_oauth_signature = urllib.parse.quote(oauth_signature, safe='')

In [8]:
body = {'grant_type' : '{}'.format(grant_type)}

headers = {
            'Content-Type' : 'application/x-www-form-urlencoded',
            'Authorization' : 'OAuth oauth_consumer_key="{0}",oauth_nonce="{1}",oauth_signature="{2}",oauth_signature_method="HMAC-SHA256",oauth_timestamp="{3}",oauth_version="1.0"'.format(oauth_consumer_key,oauth_nonce,encoded_oauth_signature,oauth_timestamp)
          }
    
response = requests.post(url, data=body, headers=headers, verify = False)

print(response.text)

C:\Users\nanzawi\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'account.api.here.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


{"access_token":"eyJhbGciOiJSUzUxMiIsImN0eSI6IkpXVCIsImlzcyI6IkhFUkUiLCJhaWQiOiJ2TDlzcnBjREp3b2ZxMHZUcEZReSIsImlhdCI6MTY5ODA3NTkxMCwiZXhwIjoxNjk4MTYyMzEwLCJraWQiOiJqMSJ9.ZXlKaGJHY2lPaUprYVhJaUxDSmxibU1pT2lKQk1qVTJRMEpETFVoVE5URXlJbjAuLlVzV3JKZi1OalhmbzBhRGNQRDU3M1EudW5FNVc3eUJSTHhkVU1ZdEpHYWxXZzJqUFBDTWxfeXRCTTJsdEpsWTg0T0lpZzNJbFNueHdSTTdLNGhVOHlQTFBibDFsTEZURExKTjBaaFpDNGxnRlJGVER3cUFQUTNpejdKd3lERXpzZlFWVlNLSHRCdEVlMGItZVhBY09kb0NScjg5QXNyTzlJWnJTUnI5QXVtZFJjYU9XMzFBZDloUDRLQlVKdFhlVnEwLktUVGxvQ0g5T1BCOG16NXo0ZHBkWWNNQWdfVl9ackRmdkx3UkRfdGRsR2c.nWTG3N1SM8DXOGZ7QNPOnwBRBfSJmNpoELjiT-JcBfO5FKkKPQI2gs5gAcmPjahF-HPDUiYiPLaZlE2BLCb34lR4KNRG9QZ1Ae0OY46MfDOEtAnb6YTA59VMiklBHqPBRHgb88er74Auv0_W-PXJDtp2VxRsKLQqfwX3hvjbBH3iEyEXy1VWOhayZfDRMReyVQBjb_M_GhHB9e4kx3O4AC0ToRjod6M4F26TJCn1y7QKjks0p9fzqGujrnk5a86z3ZHGAEOK5HwkjVZYcT_oksDSrxWsUSjkGnqzrphOSjmgWHmN4rADks9wviFZcK0PftnsaE1zMB4c7XNoV7qYKA","token_type":"bearer","expires_in":86399}


### Example 3

In [ ]:
accesss_token = "eyJhbGciOiJSUzUxMiIsImN0eSI6IkpXVCIsImlzcyI6IkhFUkUiLCJhaWQiOiJ2TDlzcnBjREp3b2ZxMHZUcEZReSIsImlhdCI6MTY5ODA3NTkxMCwiZXhwIjoxNjk4MTYyMzEwLCJraWQiOiJqMSJ9.ZXlKaGJHY2lPaUprYVhJaUxDSmxibU1pT2lKQk1qVTJRMEpETFVoVE5URXlJbjAuLlVzV3JKZi1OalhmbzBhRGNQRDU3M1EudW5FNVc3eUJSTHhkVU1ZdEpHYWxXZzJqUFBDTWxfeXRCTTJsdEpsWTg0T0lpZzNJbFNueHdSTTdLNGhVOHlQTFBibDFsTEZURExKTjBaaFpDNGxnRlJGVER3cUFQUTNpejdKd3lERXpzZlFWVlNLSHRCdEVlMGItZVhBY09kb0NScjg5QXNyTzlJWnJTUnI5QXVtZFJjYU9XMzFBZDloUDRLQlVKdFhlVnEwLktUVGxvQ0g5T1BCOG16NXo0ZHBkWWNNQWdfVl9ackRmdkx3UkRfdGRsR2c.nWTG3N1SM8DXOGZ7QNPOnwBRBfSJmNpoELjiT-JcBfO5FKkKPQI2gs5gAcmPjahF-HPDUiYiPLaZlE2BLCb34lR4KNRG9QZ1Ae0OY46MfDOEtAnb6YTA59VMiklBHqPBRHgb88er74Auv0_W-PXJDtp2VxRsKLQqfwX3hvjbBH3iEyEXy1VWOhayZfDRMReyVQBjb_M_GhHB9e4kx3O4AC0ToRjod6M4F26TJCn1y7QKjks0p9fzqGujrnk5a86z3ZHGAEOK5HwkjVZYcT_oksDSrxWsUSjkGnqzrphOSjmgWHmN4rADks9wviFZcK0PftnsaE1zMB4c7XNoV7qYKA"

In [2]:
# Define a function to call a routing API
def call_here_routing_api(origin, destination):
    # Replace with your actual HERE API credentials
    app_id = "vL9srpcDJwofq0vTpFQy"
    app_code = "coeSXcu16kjdJAujw9INxZRAp_U"

    url = f"https://route.api.here.com/routing/7.2/calculateroute.json?waypoint0={origin}&waypoint1={destination}&mode=fastest;car;traffic:enabled&app_id={app_id}&app_code={app_code}"
    
    response = requests.get(url, verify=False)
    if response.status_code == 200:
        data = response.json().get('value', [])  
        return data
    else:
        return None

In [10]:
address = "Invalidenstr 116, 10115 Berlin, Germany"
geo = ls.geocode(query=address)
print(json.dumps(geo.to_geojson(), indent=2, sort_keys=True))

NameError: name 'ls' is not defined

### Example 4

In [ ]:
import os

from here_location_services import LS
from here_map_widget import Map, GeoJSON

LS_API_KEY = os.environ.get("coeSXcu16kjdJAujw9INxZRAp_U")  # Get API KEY from environment.
ls = LS(api_key=LS_API_KEY)

address = "Invalidenstr 116, 10115 Berlin, Germany"
gc_response = ls.geocode(query=address)

data = gc_response.to_geojson()
geo_layer = GeoJSON(data=data)

m = Map(api_key=LS_API_KEY, center=[52.53086, 13.38469], zoom=12)
m.add_layer(geo_layer)
m